# Install Libraries

In [ ]:
!pip install opencv-python
!pip install imutils
!pip install tensorflow

# Import Libraries

In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

import os 
# path to F1_mnist_utils
cwd = os.getcwd() 
cwd = cwd+'/FL & Darthmouth/'
import sys  
sys.path.insert(0, cwd)

from fl_mnist_implementation_tutorial_utils import *

# Data Set up

## Path to Data & Loading Data

In [ ]:
# Path to Data Data Path
cwd = os.getcwd() #Path to current data and program files
img_path = cwd+'/FL & Darthmouth/MNIST Data/archive/trainingSet/trainingSet'

#Path list to all data using the path object form imutils
image_paths = list(paths.list_images(img_path))

#apply our function
image_list, label_list = load(image_paths, verbose=10000)

#binarize the labels
lb = LabelBinarizer() # 1-hot-encode the labels 
label_list = lb.fit_transform(label_list) # convcvers labels to bionary



[INFO] processed 10000/42000
[INFO] processed 20000/42000
[INFO] processed 30000/42000
[INFO] processed 40000/42000


## Test-Train Split

In [ ]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list, 
                                                    label_list, 
                                                    test_size=0.1, 
                                                    random_state=42)

In [ ]:
print(label_list)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


# Federated Learning Set-up

## Create Clients

In [ ]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

## Batch Cliets

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [ ]:
print(test_batched)

<BatchDataset shapes: ((None, 784), (None, 10)), types: (tf.float64, tf.int32)>


## 3 Layer MLP (multi-layer perception) model

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

## Declearing model optimizer, loss function and a metric

In [ ]:
#Learning Rate
lr = 0.01 

# number global epochs (aggregations)
comms_round = 100

#Loss Function
loss='categorical_crossentropy'

#Evaluation Metric
metrics = ['accuracy']

#Model optimization using SGD (Stochastic Gradient Descent)
optimizer = SGD(learning_rate=lr, decay=lr / comms_round, momentum=0.9)              

# Model Aggregation (Federated Averaging) and Testing 

In [ ]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 85.881% | global_loss: 1.7663754224777222
comm_round: 1 | global_acc: 89.262% | global_loss: 1.6617894172668457
comm_round: 2 | global_acc: 90.500% | global_loss: 1.629312515258789
comm_round: 3 | global_acc: 91.071% | global_loss: 1.6098856925964355
comm_round: 4 | global_acc: 91.405% | global_loss: 1.601041555404663
comm_round: 5 | global_acc: 91.952% | global_loss: 1.591377854347229
comm_round: 6 | global_acc: 92.405% | global_loss: 1.5863757133483887
comm_round: 7 | global_acc: 92.786% | global_loss: 1.5805827379226685
comm_round: 8 | global_acc: 93.048% | global_loss: 1.5760444402694702
comm_round: 9 | global_acc: 93.214% | global_loss: 1.5733847618103027
comm_round: 10 | global_acc: 93.548% | global_loss: 1.5686380863189697
comm_round: 11 | global_acc: 93.833% | global_loss: 1.56605863571167
comm_round: 12 | global_acc: 93.810% | global_loss: 1.5632315874099731
comm_round: 13 | global_acc: 94.071% | global_loss: 1.5612705945968628
comm_round: 14 | glob

# SGD Comparison to Federated Learning Accuracy

In [ ]:
#Creating SGD Dataset with all hyperparameters equal to that of FL, however with a batch size of 320 as there are not multiple independent models or multiple clients to have said models
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320) 

#Initializing Model
smlp_SGD = SimpleMLP()

#Generating model with a shape of 784, and 10 classes
SGD_model = smlp_SGD.build(784, 10) 

#Compilation of Model using model paramaters
SGD_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

comm_round: 1 | global_acc: 94.810% | global_loss: 1.5470417737960815
